In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, Draw

from chembl_webresource_client.new_client import new_client
from chembl_webresource_client.utils import utils

import numpy as np
import json as json
import csv
import requests
import xml.etree.cElementTree as et

from Bio.SeqUtils.ProtParam import ProteinAnalysis

import random

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn import tree
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import roc_auc_score
import time
from sklearn.svm import SVC
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

# Data Collection

## Positive Data

### Request Raw Data From ChEMBL Dataset

In [2]:
activities = new_client.activity
drugs = new_client.drug
targets = new_client.target
FOLDER = 'data\\'

dic = activities[:500]
print('You may need to wait for more than an hour, please be paitent :-)')
frame = pd.DataFrame(dic)
print('Thanks for the waiting, now you can access the activity data in the dataframe named \'frame\'')

You may need to wait for more than an hour, please be paitent :-)
Thanks for the waiting, now you can access the activity data in the dataframe named 'frame'


### Clean the dataset

In [3]:
def remove_invalid_comment(data):
    valid_rows = data['data_validity_comment'].isnull()
    df = data[valid_rows]
    return df

def remove_non_uniprots(data):
    data['mark_to_remove'] = False
    for i in range(data.shape[0]):
        target_id = data['target_chembl_id'][i]
        targets1 = new_client.target.filter(target_chembl_id__in=target_id) 
        uniprots = []
        uniprots = sum([[comp['accession'] for comp in t['target_components']] for t in targets1],[])
        if(len(uniprots)==0 or uniprots[0] is None):
            data['mark_to_remove'][i] = True
    valid_rows = data['mark_to_remove'] == False
    df = data[valid_rows]
    return df

def remove_non_smile(data):
    data['mark_to_remove'] = False
    for i in range(data.shape[0]):
        if(data['canonical_smiles'][i] is None):
            data['mark_to_remove'][i] = True
    valid_rows = data['mark_to_remove'] == False
    df = data[valid_rows]
    return df

def clean_dataset(data):
    data = remove_invalid_comment(data)
    print('invaild comment data removed')
    data = data.set_index(np.arange(0,data.shape[0]))
    data = remove_non_smile(data)
    data = data.set_index(np.arange(0,data.shape[0]))
    data = remove_non_uniprots(data)
    print('Invaild uniprot data removed')
    return data

df_clean = clean_dataset(frame)
df_clean = df_clean.set_index(np.arange(0,df_clean.shape[0]))
print('The positive data set includes %s pieces of data' %(df_clean.shape[0]))

invaild comment data removed


D:\Users\28297\Anaconda2\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Users\28297\Anaconda2\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Invaild uniprot data removed
The positive data set includes 237 pieces of data


## Negative Data

### Randomly select target data

In [4]:
# get random target data
target_list = []
for i in range(500):
    num = random.randint(0,len(targets))
    # get a random target id
    ctid = targets[num]['target_chembl_id']
    target_list.append(ctid)
    negative_target_df = pd.DataFrame(target_list, columns=['target_chembl_id'])

    negative_target_df = remove_non_uniprots(negative_target_df)
    negative_target_df = negative_target_df.set_index(np.arange(0,negative_target_df.shape[0]))
    if((i+1)%10 == 0):
        print('%s negative target data has been selected' %((i+1)))
    if(negative_target_df.shape[0] == df_clean.shape[0]):
        break

negative_target_df = negative_target_df.drop(['mark_to_remove'],axis = 1)
print('The negative target data has been saved in the dataframe named\'negative_target_df\'')

D:\Users\28297\Anaconda2\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


10 negative target data has been selected
20 negative target data has been selected
30 negative target data has been selected
40 negative target data has been selected
50 negative target data has been selected
60 negative target data has been selected
70 negative target data has been selected
80 negative target data has been selected
90 negative target data has been selected
100 negative target data has been selected
110 negative target data has been selected
120 negative target data has been selected
130 negative target data has been selected
140 negative target data has been selected
150 negative target data has been selected
160 negative target data has been selected
170 negative target data has been selected
180 negative target data has been selected
190 negative target data has been selected
200 negative target data has been selected
210 negative target data has been selected
220 negative target data has been selected
230 negative target data has been selected
240 negative target 

### Random Selection for molecule data

In [5]:
# molecule data
from chembl_webresource_client.new_client import new_client
molecules = new_client.molecule

df_molecules = pd.DataFrame(molecules[:500])

# make sure the data is negative
def data_is_negative(molecule_id,target_id):
    activities = new_client.activity.filter(molecule_chembl_id__in=molecule_id)
    data_negativity = True
    for dic in activities:
        if(dic['target_chembl_id'] == target_id):
            data_negativity = False
            break
    return data_negativity

# # see if the selected molecules has a SMILE
def smile_exist(mole):
    smile_existence = None
    
    if(mole['molecule_structures'] == None):
        smile_existence = False
    elif('canonical_smiles' in mole['molecule_structures'].keys()):
        smile_existence = True
    else:
        smile_existence = False
    return smile_existence

def structure_exist(mole):
    existence = None
    if('molecule_structures' in mole.keys()):
        existence = smile_exist(mole)
    else:
        existence = False
    
    return existence

def get_mol_data(num):
    cdid = molecules[num]['molecule_chembl_id']
    ctid = negative_target_df['target_chembl_id'][i]
    data_negativity = data_is_negative(cdid,ctid)
    existence = structure_exist(molecules[num])
    return cdid,ctid,data_negativity,existence

# get random drug data
drug_list = []
for i in range(500):
    num = random.randint(0,len(molecules))
    if(molecules[num] is None):
        num = 0
    cdid,ctid,data_negativity,existence = get_mol_data(num)
    
    while((data_negativity == False) or (existence == False)):
        new_num = random.randint(0,len(molecules))
        cdid,ctid,data_negativity,existence = get_mol_data(new_num)
        num = new_num
        
    structure = molecules[num]['molecule_structures']
    smile = structure['canonical_smiles']
    dic = {'molecule_chembl_id':cdid,'smile':smile}
    drug_list.append(dic)
    if((i+1)%10 == 0):
        print('%s negative molecule data has been selected' %((i+1)))
    if(len(drug_list) == df_clean.shape[0]):
        break
    
negative_molecule_df = pd.DataFrame(drug_list)
print('The negative target data has been saved in the dataframe named\'negative_molecule_df\'')

10 negative molecule data has been selected
20 negative molecule data has been selected
30 negative molecule data has been selected
40 negative molecule data has been selected
50 negative molecule data has been selected
60 negative molecule data has been selected
70 negative molecule data has been selected
80 negative molecule data has been selected
90 negative molecule data has been selected
100 negative molecule data has been selected
110 negative molecule data has been selected
120 negative molecule data has been selected
130 negative molecule data has been selected
140 negative molecule data has been selected
150 negative molecule data has been selected
160 negative molecule data has been selected
170 negative molecule data has been selected
180 negative molecule data has been selected
190 negative molecule data has been selected
200 negative molecule data has been selected
210 negative molecule data has been selected
220 negative molecule data has been selected
230 negative molecu

# Feature Extraction

## Positive Data

### Request Uniprot ids

In [6]:
def get_uniprot_id(target_id):
    targets = new_client.target.filter(target_chembl_id__in=target_id) 
    uniprots = []
    uniprots = sum([[comp['accession'] for comp in t['target_components']] for t in targets],[])
    uniprot_id = uniprots[0]
    return uniprot_id

def get_all_uniprot_id(data,file_name = 'uniprot_id.txt'):
    targets = new_client.target
    target_list = []
    dumped_id = []
    for i in range(0, data.shape[0]):
        if((i+1)%10 == 0):
            print('%s pieces of data have been retrieved' %((i+1)))
        target_id = data['target_chembl_id'][i]
        uniprot_id = get_uniprot_id(target_id)
        target_list.append(uniprot_id)
    df_tmp = pd.DataFrame(target_list, columns = ['uniprot_id'])
    # write the IDs into a txt file
    df_tmp.to_csv(FOLDER + file_name, sep='\t', index=False, header = None)
    print('The uniprot id file can be found as \'%s\' in the folder \'data\'' %(file_name))
get_all_uniprot_id(df_clean)

10 pieces of data have been retrieved
20 pieces of data have been retrieved
30 pieces of data have been retrieved
40 pieces of data have been retrieved
50 pieces of data have been retrieved
60 pieces of data have been retrieved
70 pieces of data have been retrieved
80 pieces of data have been retrieved
90 pieces of data have been retrieved
100 pieces of data have been retrieved
110 pieces of data have been retrieved
120 pieces of data have been retrieved
130 pieces of data have been retrieved
140 pieces of data have been retrieved
150 pieces of data have been retrieved
160 pieces of data have been retrieved
170 pieces of data have been retrieved
180 pieces of data have been retrieved
190 pieces of data have been retrieved
200 pieces of data have been retrieved
210 pieces of data have been retrieved
220 pieces of data have been retrieved
230 pieces of data have been retrieved
The uniprot id file can be found as 'uniprot_id.txt' in the folder 'data'


In [7]:
# retrieve the target data
df_tmp = df_clean['target_chembl_id']
df_tmp.to_csv(FOLDER + 'target_chembl_id.txt', sep='\t', index=False, header = None)
print('The target chembl id file can be found as \'%s\' in the folder \'data\'' %('target_chembl_id.txt'))

The target chembl id file can be found as 'target_chembl_id.txt' in the folder 'data'


D:\Users\28297\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


### Access protein sequences from Uniprot

In [8]:
# get file from uniprot
def get_files(resource):
    # Uniprot webservice
    sequence_file = requests.get('http://www.uniprot.org/uniprot/' + resource)

    # If response is empty
    if len(sequence_file.text) == 0:
        print 'not available in .xml or does not exist'
        sequence_file = requests.get('http://www.uniprot.org/uniprot/' + 'A0ZX81.xml')
        with open(FOLDER + resource, "wb") as file_name:
            [file_name.write(line + '\n') for line in sequence_file.iter_lines()]
        

    # http not 200
    elif sequence_file.status_code != 200:
        print('http error ' + str(sequence_file.status_code))
        sequence_file = requests.get('http://www.uniprot.org/uniprot/' + 'A0ZX81.xml')
        with open(FOLDER + resource, "wb") as file_name:
            [file_name.write(line + '\n') for line in sequence_file.iter_lines()]

    # If response is html, then it's invalid
    else:
        html = False
        for line in sequence_file.iter_lines():
            if '<!DOCTYPE html' in line:
                print 'not available in .' + output_format + ' or does not exist'
                html = True

        with open(FOLDER + resource, "wb") as file_name:
            [file_name.write(line + '\n') for line in sequence_file.iter_lines()]

# get one protein sequence
def get_protein_seq(file_name):
    tree = et.ElementTree(file = FOLDER + file_name)
    root = tree.getroot()
    for child in root:
        for grchild in child:
            if grchild.tag == '{http://uniprot.org/uniprot}sequence':
                seq = grchild.text.strip()
                return seq

# get all protein sequences
def get_all_protseq(file_name,target_chembl_id = 'target_chembl_id.txt'):
    id_list = []
    with open(FOLDER + file_name, 'r') as f:
        data = csv.reader(f)
        for row in f:
            id_list.append(row.strip())
        
    seq_list = []
    count = 1
    for i in id_list:
        resource = i + '.xml'
        get_files(resource)
        seq = get_protein_seq(resource)
        seq_list.append(seq.replace('\n',''))
        if(count%10 == 0):
            print('%s sequences have been extracted' %(count))
        count+=1
    chembl_id_list = []
    with open(FOLDER + target_chembl_id,'r') as f1:
        data1 = csv.reader(f1)
        for row in f1:
            chembl_id_list.append(row.strip())
    tmp = {'uniprot_id':id_list,
          'sequence':seq_list,
          'target_id':chembl_id_list}
    df = pd.DataFrame(tmp)
    return df

protein_df = get_all_protseq('uniprot_id.txt')
print('The protein information is saved in the dataframe named \'protein_df\'')

10 sequences have been extracted
20 sequences have been extracted
30 sequences have been extracted
40 sequences have been extracted
50 sequences have been extracted
60 sequences have been extracted
70 sequences have been extracted
80 sequences have been extracted
90 sequences have been extracted
100 sequences have been extracted
110 sequences have been extracted
120 sequences have been extracted
130 sequences have been extracted
140 sequences have been extracted
150 sequences have been extracted
160 sequences have been extracted
170 sequences have been extracted
180 sequences have been extracted
190 sequences have been extracted
200 sequences have been extracted
210 sequences have been extracted
220 sequences have been extracted
230 sequences have been extracted
The protein information is saved in the dataframe named 'protein_df'


### Compute Morgan Fingerprints

In [9]:
# get molecule id and smiles
data = {'molecule_chembl_id':df_clean['molecule_chembl_id'],
       'smile':df_clean['canonical_smiles']}
molecule_id_df = pd.DataFrame(data)

# compute morgan fingerprint
def morgan_fingerprint(smile,radius, n_bits):
    mol = Chem.MolFromSmiles(smile)
    vect = AllChem.GetHashedMorganFingerprint(mol = mol, radius = radius, nBits = n_bits)
    vect = vect.GetNonzeroElements()
    vect_keys = list(vect.keys()) # get the non-zero keys
    vect_values = list(vect.values()) # get the non-zero values
    
    vect_dense = np.zeros(shape = (n_bits,))
    vect_dense[vect_keys] = vect_values
    fingerprint = ''
    for i in vect_dense:
        fingerprint += str(int(i))
    return fingerprint

# compute all morgan fingerprint
def all_morgan_fingerprint(data):
    df_list = []
    for i in range(data.shape[0]):
        fingerprint = morgan_fingerprint(data['smile'][i],2,300)
        df_list.append(fingerprint)
        if((i+1)%10 == 0):
            print('%s fingerprints have been calculated' %((i+1)))
    df = pd.DataFrame(df_list, columns = ['morgan_fingerprint'])
    df_new = pd.concat([data,df],axis = 1)
    return df_new 

molecule_df = all_morgan_fingerprint(molecule_id_df)
print('The computation of fingerprints is done')

10 fingerprints have been calculated
20 fingerprints have been calculated
30 fingerprints have been calculated
40 fingerprints have been calculated
50 fingerprints have been calculated
60 fingerprints have been calculated
70 fingerprints have been calculated
80 fingerprints have been calculated
90 fingerprints have been calculated
100 fingerprints have been calculated
110 fingerprints have been calculated
120 fingerprints have been calculated
130 fingerprints have been calculated
140 fingerprints have been calculated
150 fingerprints have been calculated
160 fingerprints have been calculated
170 fingerprints have been calculated
180 fingerprints have been calculated
190 fingerprints have been calculated
200 fingerprints have been calculated
210 fingerprints have been calculated
220 fingerprints have been calculated
230 fingerprints have been calculated
The computation of fingerprints is done


In [10]:
positive_data = pd.concat([molecule_df,protein_df],axis = 1)
positive_data.to_csv(FOLDER+'positive_features.csv')
print('The positive features are saved in the file \'positive_features.csv\'')

The positive features are saved in the file 'positive_features.csv'


## Negative Data

### Target Data

In [11]:
negative_target_df = negative_target_df.set_index(np.arange(0,negative_target_df.shape[0]))
negative_target_df.to_csv(FOLDER + 'negative_target_chembl_id.txt', sep='\t', index=False, header = None)
get_all_uniprot_id(negative_target_df,'negative_uniprot_id.txt')
negative_target_df.to_csv(FOLDER + 'negative_target_chembl_id.txt', sep='\t', index=False, header = None)
print('The negative target data has been save as \'negative_target_chembl.txt\'in the folder \'data\'')
# get protein sequence dataframe
negative_protseq_df = get_all_protseq('negative_uniprot_id.txt','negative_target_chembl_id.txt')

10 pieces of data have been retrieved
20 pieces of data have been retrieved
30 pieces of data have been retrieved
40 pieces of data have been retrieved
50 pieces of data have been retrieved
60 pieces of data have been retrieved
70 pieces of data have been retrieved
80 pieces of data have been retrieved
90 pieces of data have been retrieved
100 pieces of data have been retrieved
110 pieces of data have been retrieved
120 pieces of data have been retrieved
130 pieces of data have been retrieved
140 pieces of data have been retrieved
150 pieces of data have been retrieved
160 pieces of data have been retrieved
170 pieces of data have been retrieved
180 pieces of data have been retrieved
190 pieces of data have been retrieved
200 pieces of data have been retrieved
210 pieces of data have been retrieved
220 pieces of data have been retrieved
230 pieces of data have been retrieved
The uniprot id file can be found as 'negative_uniprot_id.txt' in the folder 'data'
The negative target data has 

### Molecule Data

In [12]:
# compute molecule morgan fingerprint
negative_drug_df = all_morgan_fingerprint(negative_molecule_df)
negative_data = pd.concat([negative_drug_df,negative_protseq_df],axis = 1)
negative_data.to_csv(FOLDER + 'negative_features.csv')
print('The positive features are saved in the file \'negative_features.csv\'')

10 fingerprints have been calculated
20 fingerprints have been calculated
30 fingerprints have been calculated
40 fingerprints have been calculated
50 fingerprints have been calculated
60 fingerprints have been calculated
70 fingerprints have been calculated
80 fingerprints have been calculated
90 fingerprints have been calculated
100 fingerprints have been calculated
110 fingerprints have been calculated
120 fingerprints have been calculated
130 fingerprints have been calculated
140 fingerprints have been calculated
150 fingerprints have been calculated
160 fingerprints have been calculated
170 fingerprints have been calculated
180 fingerprints have been calculated
190 fingerprints have been calculated
200 fingerprints have been calculated
210 fingerprints have been calculated
220 fingerprints have been calculated
230 fingerprints have been calculated
The positive features are saved in the file 'negative_features.csv'


### Amino Acids Compositions

In [13]:
# read the file
positive_d = pd.read_csv(FOLDER+'positive_features.csv')
negative_d = pd.read_csv(FOLDER+'negative_features.csv')

positive_d['y'] = 1
negative_d['y'] = 0

data = pd.concat([positive_d,negative_d],axis = 0)
data = data.drop(['Unnamed: 0'], axis = 1)
data = data.set_index(np.arange(0,data.shape[0]))

In [14]:
# construct bigram
def construct_bigram(seq): # amino acids composition
    a='ACDEFGHIKLMNPQRSTVWY'
    b=[]
    for i in range(0,len(a)):
        for j in range(0,len(a)):
            b.append(a[i]+a[j])

    bi_dict=dict((letter,seq.count(letter)) for letter in set(b))
    bi=list(bi_dict.values())
    return bi

def protein_features(seq):
    sequence = []
    analysed_seq = ProteinAnalysis(seq)
    sequence.append(seq)
    
    unigram = analysed_seq.count_amino_acids() # count single amino acid
    uni = list(unigram.values())
    
    # construct bigram
    bi = construct_bigram(seq) 
    
    iso=analysed_seq.isoelectric_point() # count iso

    X_list=[0]*421
    X_list[0] = iso
    X_list[1:21]=uni[:]
    X_list[21:421]=bi[:]

    return X_list

def compute_all_features(data,features):
    data_list = []
    for i in range(data.shape[0]):
        seq = data['sequence'][i]
        protf = protein_features(seq)
        data_list.append(protf)
        if((i+1)%10 == 0):
            print('%s protein features have been calculated' %((i+1)))
    df = pd.DataFrame(data_list)
    df_new = pd.concat([features,df],axis = 1)
    return df_new

In [15]:
feature_data = pd.DataFrame()
feature_data = compute_all_features(data,feature_data)
feature_data['y'] = data['y']
feature_data = feature_data.dropna(axis=1,how='any')

feature_data.to_csv(FOLDER + 'feature_data.csv')
print('The feature data has been saved into file named \'feature_data.csv\'')

10 protein features have been calculated
20 protein features have been calculated
30 protein features have been calculated
40 protein features have been calculated
50 protein features have been calculated
60 protein features have been calculated
70 protein features have been calculated
80 protein features have been calculated
90 protein features have been calculated
100 protein features have been calculated
110 protein features have been calculated
120 protein features have been calculated
130 protein features have been calculated
140 protein features have been calculated
150 protein features have been calculated
160 protein features have been calculated
170 protein features have been calculated
180 protein features have been calculated
190 protein features have been calculated
200 protein features have been calculated
210 protein features have been calculated
220 protein features have been calculated
230 protein features have been calculated
240 protein features have been calculated
2

# Neural Network

In [16]:
feature_data = pd.read_csv(FOLDER+'feature_data.csv')

## Split the dataset

In [17]:
trainset, testset = train_test_split(feature_data, test_size=0.2, random_state=0)
X_train = trainset.drop(['y'], axis = 1)
y_train = trainset['y']
X_test = testset.drop(['y'], axis = 1)
y_test = testset['y']

## Training

In [22]:
clf = MLPClassifier(hidden_layer_sizes=(500,20), max_iter=i)
clf.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(500, 20), learning_rate='constant',
       learning_rate_init=0.001, max_iter=236, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

# Model Evaluation

In [23]:
y_pred = clf.predict(X_test)
print(y_pred)

[1 1 0 1 0 0 1 0 0 0 1 0 0 1 1 0 0 1 0 0 1 0 0 1 1 0 0 1 0 1 0 1 0 1 1 0 1
 1 0 0 1 1 0 1 1 0 0 1 1 1 1 1 1 0 0 1 0 1 0 0 0 1 0 1 1 1 0 1 0 1 0 1 1 1
 0 1 1 1 1 1 0 0 1 0 1 1 0 0 1 0 0 0 1 1 0]
